# Normalization

- Correcting spelling using model https://huggingface.co/oliverguhr/spelling-correction-english-base
    - this produces too many mistakes
- expanding contractions
- removing punctuation & capitalization


In [7]:
import sqlite3

import pandas as pd

conn = sqlite3.connect("../../../giicg.db")
prompts = pd.read_sql("Select * from manually_corrected_prompts", conn)
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
755,724,31,user,import pandas as pd\nimport numpy as np\nfrom ...,Please replace my retrieval pipeline here with...,import pandas as pd\nimport numpy as np\nfrom ...,You are tasked with separating user prompts in...,Man (cisgender),92,en
756,726,31,user,"please update my code accordingly, no comments...","please update my code accordingly, no comments...",,,Man (cisgender),92,en
757,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,"I want to tune optimal thresholds. Currently, ...",import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
758,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",I want to use an LLM for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en


# Normalization Steps

In [8]:
from helpers.normalization import expand_contractions

prompts['conversational'] = prompts['conversational'].apply(expand_contractions)
prompts


,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
755,724,31,user,import pandas as pd\nimport numpy as np\nfrom ...,Please replace my retrieval pipeline here with...,import pandas as pd\nimport numpy as np\nfrom ...,You are tasked with separating user prompts in...,Man (cisgender),92,en
756,726,31,user,"please update my code accordingly, no comments...","please update my code accordingly, no comments...",,,Man (cisgender),92,en
757,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,"I want to tune optimal thresholds. Currently, ...",import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
758,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",I want to use an LLM for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en


### Spell Correction Bad Examples

In [10]:
from helpers.normalization import load_spell_correction_model, correct_spelling

tokenizer, model, device = load_spell_correction_model()
text = "hey, I\'m writing an application. Because I want to include some sound snippets, I decided to send them as an application.html file, a style.css and a folder with resources (sound, img, font). Please place the .html and .css document in such a way that it makes sense for this purpose. The application.html file should be structured quite simply - no header, only a body that contains the headline and a flow text. In between then audioplayer and at the end images. It should remember a classic .pdf document -- just interactive thanks to html5."

#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

#text = "i load it like this:"
#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

#text = "ok nice. can i let the color switch already on hover? i.e. when im on the work page, and move hover over contact, then work becomes 3e3e3e and contact becomes blue"
#text = (expand_contractions(text))
#print(text)
#print(correct_spelling(text, tokenizer, model, device))

text = "inside a <figure>?"
text = (expand_contractions(text))
print(text)
print(correct_spelling(text, tokenizer, model, device))

model loaded
inside a <figure>?
Inside a computer, what?


In [6]:
from helpers.normalization import remove_punct_cap

prompts_punct_cap = prompts.copy()
prompts_punct_cap['conversational'] = prompts_punct_cap['conversational'].apply(remove_punct_cap)
prompts_punct_cap

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language,normalized
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en,parsing data from python iterator how it could...
1,724,31,user,Write python function to do operations with in...,Write python function to do operations with in...,,,Man (cisgender),6,en,write python function to do operations with in...
2,1124,53,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en,write shortest tutorial on creating rag on ema...
3,1126,53,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en,what is faiss
4,1128,53,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,,Man (cisgender),6,en,transform given code to process large 32mbox file
...,...,...,...,...,...,...,...,...,...,...,...
701,1114,52,user,"ok and then last question: Why is ""Consolas"" l...","ok and then last question: Why is ""Consolas"" l...",,,Man (cisgender),91,en,ok and then last question why is consolas load...
702,1116,52,user,i load it like this:\n\n@font-face {\n font...,i load it like this:,@font-face {\n font-family: 'Consolas';\n ...,,Man (cisgender),91,en,i load it like this
703,1118,52,user,should i be able to open the .woff in windows ...,should i be able to open the .woff in windows ...,,,Man (cisgender),91,en,should i be able to open the window in windows...
704,1120,52,user,"ok, i am not calling it in the body. how do i ...","ok, i am not calling it in the body. how do i ...",,,Man (cisgender),91,en,ok i am not calling it in the body how do i ca...


# Write back to Database

In [9]:
prompts.to_sql('expanded_prompts', conn, if_exists='replace', index=False)
#prompts_punct_cap.to_sql('punct_cap_prompts', conn, if_exists='replace', index=False)
conn.close()